Скрипт для создания графа друзей в вк из матрицы ребер. Она и создается этим скриптом и сохраняется в csv.
Здесь используется VK API. Подробнее о нём вы можете узнать по [ссылке](https://vk.com/dev/first_guide). Там вам нужно получить свой токен и пользоваться им здесь. Никому не давайте свой токен!

In [ ]:
# подгружаем пакеты
import requests
import pandas as pd
from tqdm import tqdm_notebook
import time

In [ ]:
# уникальный токен
token = 'ваш токен'

In [ ]:
def request_get_friends(id = '', fields = [], token = token):
    """Возращает информацию по друзьям для опредленного человека"""
    r = 'https://api.vk.com/method/friends.get?' + 'user_id=' + str(id) + '&fields=' + ','.join(fields) + '&v=5.92&access_token=' + token
    return r

In [ ]:
# список моих друзей
r = request_get_friends(id = 73614108,fields=['first_name'])
my_friends = requests.get(r).json()['response']['items']
id_friends = [x['id'] for x in my_friends]

In [ ]:
# mf это матрица ребер для моих друзей
id_friends = [x['id'] for x in my_friends]
mf = pd.DataFrame(my_friends)
mf['id_1'] = 73614108
mf['name_1'] = 'Зарманбетов Ахмед'
mf['id_2'] = mf['id']
mf['name_2'] = mf[['last_name', 'first_name']].apply(lambda x: ' '.join(x), axis=1)
mf = mf.loc[:,['id_1', 'name_1','id_2','name_2']]

final = mf.copy()

In [ ]:
for i in tqdm_notebook(range(mf.shape[0])):
    t = request_get_friends(id = mf.iloc[i,2], fields=['first_name'])
    t1 = requests.get(t).json()
    if 'response' in t1.keys():
        t1 = t1['response']['items']
        t2 = pd.DataFrame(t1)
        t2['id_1'] = mf.iloc[i,2]
        t2['name_1'] = mf.iloc[i,3]
        t2['id_2'] = t2['id']
        t2['name_2'] = t2[['last_name', 'first_name']].apply(lambda x: ' '.join(x), axis=1)
        t2 = t2.loc[:,['id_1', 'name_1','id_2','name_2']]
        t2 = t2[t2['id_2'].isin(id_friends)]
        final = pd.concat([final, t2])
        time.sleep(0.5)

In [ ]:
final.shape

In [ ]:
# удаляем дубликаты
final = pd.DataFrame([x if x[0]>x[2] else [x[2],x[3],x[0],x[1]] for x in final.values], columns=['id_1', 'name_1', 'id_2', 'name_2'])
final = final.drop_duplicates() 

In [ ]:
final.shape

In [ ]:
final.to_csv('edges.csv', index = False)